In [1]:
# Obtaining the list of ingredients for each recipe. Only interested
# in the ingredients, and so I've stripped the other information such as URL
import json

with open('combinedRecipes.JSON') as recipes:
    data = json.load(recipes)
# print(data)

ingrs_per_recipe = []
for obj in data:
    ingrs_per_recipe.append(obj["ingredients"])
print(ingrs_per_recipe)

[['clove', 'cloves', 'garlic', 'cumin', 'tea', 'pepper', 'oregano', 'orange', 'oranges', 'lime', 'limes', 'olive', 'olive oil', 'salt', 'pork', 'mint', 'bean', 'beans', 'black bean', 'black beans', 'rice'], ['ponzu', 'shallot', 'mustard', 'olive', 'olive oil', 'wasabi', 'steak', 'tea', 'cucumber', 'salt'], ['rice', 'rice vinegar', 'vinegar', 'lemon', 'lime', 'lime juice', 'soy sauce'], ['paprika', 'brown sugar', 'sugar', 'salt', 'mustard', 'seed', 'garlic', 'garlic powder', 'onion', 'onion powder', 'oregano', 'coriander', 'cumin', 'tea', 'pepper', 'red pepper', 'red pepper flakes', 'ketchup', 'molasses', 'worcestershire sauce', 'apple', 'apple cider vinegar', 'cider', 'vinegar', 'pork'], ['flour', 'sugar', 'salt', 'tea', 'butter', 'water'], ['pepper', 'poblano', 'poblano pepper', 'potato', 'salt', 'vinegar', 'white vinegar', 'onion', 'red onion', 'tea', 'thyme', 'cilantro', 'parsley'], ['lemon', 'lemons', 'sugar', 'tea'], ['lemon', 'lemons', 'sugar', 'water'], ['water', 'tea'], ['bean'

In [2]:
# getting the list of ingredients, lowercasing, uniqifying, then sorting alphabetically
with open('foodList.txt') as f:
        ing_list = f.readlines()
ing_list = [ing[:-1].lower() for ing in ing_list]
uniq_ing_list = list(set(ing_list)) # had 3 duplicates
uniq_ing_list = sorted(uniq_ing_list)
print(len(uniq_ing_list))
# print(uniq_ing_list)
# print('fennel seeds' in uniq_ing_list)





1024


In [3]:
import pandas as pd
import itertools as iter
import numpy as np

columns = uniq_ing_list
print(len(columns))
df = pd.DataFrame(0, index=columns, columns=columns)
count = 0

for ingredient_list in ingrs_per_recipe:
    for pair in list(iter.permutations(ingredient_list,2)):
        try: 
            count += 1
            r = pair[0]
            c = pair[1]
            curr_value = df.loc[r, c]
            new_value = curr_value + 1
            df.ix[r, c] = new_value
        except KeyError as key_err:
            print('Could not find key in foodList:', key_err)

# diagonal should be set to 0 because no such case that 
# apples will be paired with apples
df.values[[np.arange(len(columns))]*2] = 0



1024


In [ ]:
#
# Sanity checks: 
#
# 1. check for symmetry A^T = A
# np.diag(df).sum() # -> 0
# dfT = df.transpose()
# print((dfT == df).all().sum()) # if sums to len(columns), then it is symmetric

# 2. check if row and col labels are the same
# list(df.columns.values) == list(df.index.values) -> true


In [4]:
#
# Manipulate data
#
# Now need to remove row and columns that have all 0s
df = df.loc[:, (df != 0).any(axis=0)]
df = df.loc[(df != 0).any(axis=1), :]
print(df.shape) # -> (371, 371)

# convert values to probability
all = df.sum().sum() # total sum of df
print('TOTAL', all)
df = df.divide(float(-all)).add(1)
df = df.replace(1.0, 0.0)
print(df)

# get new column names, will need later
new_cols = list(df.columns.values)
print(new_cols)

(371, 371)
TOTAL 50390
                       absinthe    adobo      ale  allspice    almond  \
absinthe                    0.0  0.00000  0.00000   0.00000  0.000000   
adobo                       0.0  0.00000  0.00000   0.00000  0.000000   
ale                         0.0  0.00000  0.00000   0.00000  0.000000   
allspice                    0.0  0.00000  0.00000   0.00000  0.000000   
almond                      0.0  0.00000  0.00000   0.00000  0.000000   
almond extract              0.0  0.00000  0.00000   0.00000  0.999980   
almonds                     0.0  0.00000  0.00000   0.00000  0.999921   
anchovy                     0.0  0.00000  0.00000   0.00000  0.000000   
anise                       0.0  0.00000  0.00000   0.00000  0.000000   
apple                       0.0  0.00000  0.99996   0.99998  0.000000   
apple cider vinegar         0.0  0.00000  0.00000   0.00000  0.000000   
apple pie spice             0.0  0.00000  0.00000   0.00000  0.000000   
apples                      

In [9]:
import math
from heapq import *

def Dijkstras(AdjMat, start):
    # Initialize single source
    dist = {}
    prev = {}
    Q = []
    for v in list(AdjMat.columns.values):
        if v == start:
            dist[v] = 0
        else:
            dist[v] = math.inf
            prev[v] = None
        heappush(Q, [ dist[v], v ])
        
    while Q:
        thru_me = heappop(Q)[1]
        for neighbor in list(AdjMat.index[(AdjMat > 0)[thru_me]]): 
            new_path_dist = dist[thru_me] + AdjMat.loc[thru_me, neighbor] # matrix is symmetric
            # Relaxtion
            if new_path_dist < dist[neighbor]:
                dist[neighbor] = new_path_dist
                prev[neighbor] = thru_me
                # Decrease priority
                for node in Q:
                    if node[1] == neighbor:
                        node[0] = new_path_dist

                        
                
    return {'dist': dist, 'prev': prev}

D = Dijkstras(df, 'apple')
Dijkstra_paths = D['prev']
# D_dist = D['dist']

In [10]:
print(Dijkstra_paths)

{'currant': 'sugar', 'egg yolks': 'sugar', 'bacon': 'apple', 'seeds': 'apple', 'eggplant': 'salt', 'fennel': 'sugar', 'radishes': 'salt', 'vanilla': 'apple', 'bread': 'apple', 'seed': 'apple', 'vanilla extract': 'sugar', 'pecans': 'apple', 'unsweetened chocolate': 'sugar', 'tuna': 'salt', 'oranges': 'apple', 'olive': 'salt', 'baking soda': 'apple', 'feta': 'salt', 'sesame seed': 'salt', 'pears': 'apple', 'pepper': 'apple', 'cayenne pepper': 'apple', 'green onion': 'apple', 'basil': 'clove', 'egg whites': 'apple', 'almond extract': 'sugar', 'ice cream': 'apple', 'mint': 'sugar', 'cumin': 'apple', 'cherry': 'sugar', 'sesame seeds': 'salt', 'egg substitute': 'salt', 'pinto beans': 'salt', 'wasabi': 'salt', 'ginger': 'apple', 'kidney beans': 'water', 'squash': 'butter', 'poblano pepper': 'salt', 'chives': 'sugar', 'tortillas': 'salt', 'egg white': 'apple', 'sausages': 'salt', 'strawberry': 'salt', 'graham cracker': 'sugar', 'flour': 'apple', 'ground turkey': 'water', 'swiss cheese': 'butte

In [45]:
# print(D_prev)
# We would loop through and `start` would be different every time.
# In this case, `end` is `apple`, but would be the vertex of the outer
# for loop.

def get_path(start, end, all_paths):
    def helper(start, end, start_to_end=[]):
        start_to_end.append(start)

        if (start == end):
#             print('returned')
            return start_to_end;

#         print(start)
#         print(all_paths[start])
        next = all_paths[start]
        return helper(next, end, start_to_end)

    return {'start': start, 'end': end, 'path': helper(start, end)}
    
    
#
a_b = get_path('octopus', 'apple', Dijkstra_paths)
print(a_b)
# -> {'end': 'apple', 'path': ['octopus', 'chili', 'apple'], 'start': 'octopus'}
#
# print(get_path('apple', i, Dijkstra_paths)['path']

{'start': 'octopus', 'path': ['octopus', 'salt', 'apple'], 'end': 'apple'}


In [28]:
'apple' in Dijkstra_paths

False

In [48]:
# structure of precomputed:
# { 
#   source_i: {
#      rum: [...list of ing]
#   }
# }

# SO THIS IS THE MAIN FILE
table = {}
for end in ['apple']: # will eventually be new_cols 
    temp = {}
#     prevs = Dijkstras(df, start)[prev] #keep this, eventually it will be this
    prevs = Dijkstra_paths # this is just for now
    for i in prevs:
        temp[i] = get_path(i, end, Dijkstra_paths)['path']
    table[end] = temp
print(table)

# THEN STORE TO MONGODB
    

{'apple': {'currant': ['currant', 'sugar', 'apple'], 'tomato paste': ['tomato paste', 'salt', 'apple'], 'bacon': ['bacon', 'apple'], 'seeds': ['seeds', 'apple'], 'eggplant': ['eggplant', 'salt', 'apple'], 'fennel': ['fennel', 'sugar', 'apple'], 'radishes': ['radishes', 'salt', 'apple'], 'vanilla': ['vanilla', 'apple'], 'bread': ['bread', 'apple'], 'ketchup': ['ketchup', 'apple'], 'pecans': ['pecans', 'apple'], 'oranges': ['oranges', 'apple'], 'olive': ['olive', 'salt', 'apple'], 'baking soda': ['baking soda', 'apple'], 'feta': ['feta', 'salt', 'apple'], 'sesame seed': ['sesame seed', 'salt', 'apple'], 'pears': ['pears', 'apple'], 'pepper': ['pepper', 'apple'], 'cayenne pepper': ['cayenne pepper', 'apple'], 'green onion': ['green onion', 'apple'], 'basil': ['basil', 'clove', 'apple'], 'egg whites': ['egg whites', 'apple'], 'mushrooms': ['mushrooms', 'apple'], 'ice cream': ['ice cream', 'apple'], 'mint': ['mint', 'sugar', 'apple'], 'cumin': ['cumin', 'apple'], 'cherry': ['cherry', 'sugar

In [ ]:
test = pd.Series()

In [ ]:
import heapq as pq
h = [[5, 'write code'], [7, 'release product'], [1, 'write spec'], [3, 'create tests']]
pq.heapify(h)
pq.heappop(h)[1]
print(h)
pq.heappush(h, [2, 'create tests'])
print(h)




In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np



nparray = df.as_matrix()

plt.matshow(nparray, interpolation='none') # when more data, try without interpolation

In [ ]:
columns = ['a', 'b', 'c']
df = pd.DataFrame([[1, 0, 1], [0, 1, 0], [3, 5, 2]], index=columns, columns=columns)
print(df)

print(np.arange(3))
df.values[[np.arange(len(columns))]*2] -= 1
print(df)

print('-------------')
a = df.index[(df != 0)['c']]
for i in a:
    print(i, type(i))


In [ ]:
a=[1,2,3,1,3,2,1,1]
a = [4 if x==1 else x for x in a]
print(a)

In [ ]:
D_prev

In [25]:
tst = {}
# tst['a']['b'] = [1,2,3]
# print(tst)
a = 'a'
tst[a] = 10
print(tst)

tst['apple'] = 10
print(tst)

{'a': 10}
{'a': 10, 'apple': 10}
